In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time


In [5]:
# Function to scrape data from a given URL
def scrape_data(url):
    # Instantiate options
    options = webdriver.ChromeOptions()

    # Run browser in headless mode
    options.headless = True

    # Disable notifications and popups
    prefs = {
        "profile.default_content_setting_values.popups": 1,
    }
    options.add_experimental_option("prefs", prefs)
    options.add_argument("--disable-popup-blocking")

    # Instantiate driver
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

    # Load website
    driver.get(url)

    # Scroll and load all elements
    SCROLL_PAUSE_TIME = 1.5
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to the bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load the page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            new_height = driver.execute_script("return document.body.scrollHeight")
            break
        last_height = new_height

    # Wait for all elements to be present
    wait = WebDriverWait(driver, 10)
    wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'tfu-program-number-value')))

    # Select elements by class name
    elements = driver.find_elements(By.CLASS_NAME, 'tfu-program-number-value')
    data = []
    for element in elements:
        # Select spans within the element
        spans = element.find_elements(By.TAG_NAME, 'span')
        # Join the text of all spans and append to data list
        span_texts = ''.join(span.text for span in spans)
        data.append(span_texts)

    driver.quit()
    return data

In [14]:
us_data = scrape_data('https://www.roadscholar.org/collections/us-canada-travel-tours/')

In [18]:
er_data = scrape_data('https://www.roadscholar.org/collections/europe/')

In [23]:
la_data = scrape_data('https://www.roadscholar.org/collections/Latin-america/')

In [25]:
af_data = scrape_data('https://www.roadscholar.org/collections/africa-and-southeast-asia-tours-for-seniors/')

In [27]:
au_data = scrape_data('https://www.roadscholar.org/collections/australia-new-zealand/')

In [29]:
pl_data = scrape_data('https://www.roadscholar.org/collections/polar-regions/')